In [1]:
import numpy as np
from scipy.stats import norm

def black_scholes_greeks(S, K, T, r, sigma, q, option_type='call'):
    """
    计算欧式期权的价格及希腊字母（Delta, Gamma, Theta, Vega, Rho）
    """
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)
    n_d1 = norm.pdf(d1)
    
    # 期权价格
    if option_type == 'call':
        price = S * np.exp(-q * T) * N_d1 - K * np.exp(-r * T) * N_d2
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)
    
    # Greeks
    delta = np.exp(-q * T) * (N_d1 if option_type == 'call' else (N_d1 - 1))
    gamma = np.exp(-q * T) * n_d1 / (S * sigma * np.sqrt(T))
    vega = S * np.exp(-q * T) * n_d1 * np.sqrt(T)
    
    # Theta计算
    if option_type == 'call':
        theta = (- (S * np.exp(-q * T) * n_d1 * sigma) / (2 * np.sqrt(T)) 
                 + q * S * np.exp(-q * T) * N_d1 
                 - r * K * np.exp(-r * T) * N_d2)
    else:
        theta = (- (S * np.exp(-q * T) * n_d1 * sigma) / (2 * np.sqrt(T)) 
                 - q * S * np.exp(-q * T) * norm.cdf(-d1) 
                 + r * K * np.exp(-r * T) * norm.cdf(-d2))
    
    # Rho计算
    if option_type == 'call':
        rho = K * T * np.exp(-r * T) * N_d2
    else:
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2)
    
    return {
        'price': price,
        'delta': delta,
        'gamma': gamma,
        'theta': theta / 365,  # 转换为每日Theta
        'vega': vega,
        'rho': rho
    }


In [2]:

# 示例参数
S = 100.0    # 标的资产价格
K = 100.0    # 行权价
T = 1.0      # 到期时间（年）
r = 0.05     # 无风险利率
sigma = 0.2  # 波动率
q = 0.01     # 股息率

call_greeks = black_scholes_greeks(S, K, T, r, sigma, q, 'call')
put_greeks = black_scholes_greeks(S, K, T, r, sigma, q, 'put')

print("Call Greeks:", call_greeks)
print("Put Greeks:", put_greeks)

Call Greeks: {'price': np.float64(9.826297782739111), 'delta': np.float64(0.6117631008098845), 'gamma': np.float64(0.01887964716453252), 'theta': np.float64(-0.015703197115093385), 'vega': np.float64(37.75929432906503), 'rho': np.float64(51.35001229824934)}
Put Greeks: {'price': np.float64(5.944256857893713), 'delta': np.float64(-0.3782867329392836), 'gamma': np.float64(0.01887964716453252), 'theta': np.float64(-0.005385122351382695), 'vega': np.float64(37.75929432906503), 'rho': np.float64(-43.77293015182207)}
